In [2]:
from __future__ import division

import pandas as pd
import numpy as np
import math
from datetime import datetime,timedelta,time

%matplotlib inline

In [3]:
training_path = 'dataSets/training/'
test_path1 = 'dataSets/testing_phase1/'

In [4]:
training_traj = pd.read_csv(training_path+'trajectories(table 5)_training.csv')

In [5]:
test_traj1 = pd.read_csv(test_path1+'trajectories(table 5)_test1.csv')

In [6]:
def time_to_window(x):
    dt = datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
    dtmin = int(dt.minute / 20) * 20
    dtwindow = datetime(dt.year, dt.month, dt.day, dt.hour, dtmin, 0)
    return dtwindow

In [7]:
def format_window(wd):
    return '[{},{})'.format(str(wd), str(wd+timedelta(minutes=20)))

In [8]:
#training_traj['starting_datetime'] = pd.to_datetime(training_path['starting_time'])
training_traj['window'] = training_traj['starting_time'].apply(time_to_window)

training_traj_group_mean = \
training_traj.groupby(['intersection_id','tollgate_id','window']).mean()['travel_time']

training_traj_group_count = \
training_traj.groupby(['intersection_id','tollgate_id','window']).count()['travel_time']

In [9]:
ss = pd.read_csv('submission/submission_sample_travelTime.csv')

In [10]:
print(ss['time_window'][0])
print(format_window(training_traj['window'][0]))

[2016-10-18 08:00:00,2016-10-18 08:20:00)
[2016-07-19 00:00:00,2016-07-19 00:20:00)


In [11]:
from datetime import date, time

In [12]:
date_list = []

d1 = date(2016, 7, 19)  # start date
d2 = date(2016, 10, 17)  # end date

delta = d2 - d1         # timedelta

for i in range(delta.days + 1):
    date_list.append(d1 + timedelta(days=i))

give_time_list1 = [time(6,0), time(6,20), time(6,40), 
                   time(7,0), time(7,20), time(7,40)]
pred_time_list1 = [time(8,0), time(8,20), time(8,40), 
                   time(9,0), time(9,20), time(9,40)]

give_time_list2 = [time(15,0), time(15,20), time(15,40), 
                   time(16,0), time(16,20), time(16,40)]
pred_time_list2 = [time(17,0), time(17,20), time(17,40), 
                   time(18,0), time(18,20), time(18,40)]


In [13]:
date_list2 = []

d1 = date(2016, 10, 11)  # start date
d2 = date(2016, 10, 17)  # end date

delta = d2 - d1         # timedelta

for i in range(delta.days + 1):
    date_list2.append(d1 + timedelta(days=i))


In [14]:
# A2m.index[300].weekday()

In [15]:
def interpolate_missing_window(x):
    ss = [(i-datetime(1970,1,1)).total_seconds() for i in x.index]
    tt = x.values
    ssall = np.linspace(ss[0], ss[-1], (ss[-1]-ss[0])/1200+1)
    #print(len(ss), len(tt), len(ssall))
    ttall = np.interp(ssall, ss, tt)
    ssall = [int(i) for i in ssall]
    ssall = [datetime(1970,1,1)+timedelta(seconds=i) for i in ssall]
    return pd.Series(index=ssall, data=ttall)

In [16]:
# A2m = training_traj_group_mean.loc['A'].loc[2]

# A2m = interpolate_missing_window(A2m)


In [17]:
len(date_list)

91

In [18]:
first6 = np.array([])
next6 = np.array([])
for ii, ti in [('A',2),('A',3),('B',1),('B',3),('C',1),('C',3)]:
    mtmp = training_traj_group_mean.loc[ii].loc[ti]
    mtmp = interpolate_missing_window(mtmp)
    test6 = []
    for i in date_list:
        tmp = []
        for j in give_time_list1:
            tmp.append(mtmp[datetime.combine(i, j)])
        test6.append(tmp)
    result6 = test6
    first6 = np.append(first6, result6)
    
    test6 = []
    for i in date_list:
        tmp = []
        for j in pred_time_list1:
            tmp.append(mtmp[datetime.combine(i, j)])
        test6.append(tmp)
    result6 = test6
    next6 = np.append(next6, result6)


In [19]:
dfmorning = \
pd.concat([pd.DataFrame(first6.reshape(-1,6)),
           pd.DataFrame(next6.reshape(-1,6))], 
                        axis=1)

In [20]:
dfmorning.columns=['f0','f1','f2','f3','f4','f5',
                   'n0','n1','n2','n3','n4','n5']

In [21]:
first6 = np.array([])
next6 = np.array([])
for ii, ti in [('A',2),('A',3),('B',1),('B',3),('C',1),('C',3)]:
    mtmp = training_traj_group_mean.loc[ii].loc[ti]
    mtmp = interpolate_missing_window(mtmp)
    test6 = []
    for i in date_list:
        tmp = []
        for j in give_time_list2:
            tmp.append(mtmp[datetime.combine(i, j)])
        test6.append(tmp)
    result6 = test6
    first6 = np.append(first6, result6)
    
    test6 = []
    for i in date_list:
        tmp = []
        for j in pred_time_list2:
            tmp.append(mtmp[datetime.combine(i, j)])
        test6.append(tmp)
    result6 = test6
    next6 = np.append(next6, result6)

In [22]:
dfnight = \
pd.concat([pd.DataFrame(first6.reshape(-1,6)),
           pd.DataFrame(next6.reshape(-1,6))], 
                        axis=1)

In [23]:
dfnight.columns=['f0','f1','f2','f3','f4','f5',
                   'n0','n1','n2','n3','n4','n5']

In [48]:
len(dfmorning)

546

In [26]:
pred_loss(dfnight['f5'], dfnight['n1'])


0.24960680174029845

In [49]:
dfmorning['dn'] = 0
dfmorning['it0'] = [1]*91+[0]*455
dfmorning['it1'] = [0]*91+[1]*91+[0]*364
dfmorning['it2'] = [0]*182+[1]*91+[0]*273
dfmorning['it3'] = [0]*273+[1]*91+[0]*182
dfmorning['it4'] = [0]*364+[1]*91+[0]*91
dfnight['dn'] = 1
dfnight['it0'] = [1]*91+[0]*455
dfnight['it1'] = [0]*91+[1]*91+[0]*364
dfnight['it2'] = [0]*182+[1]*91+[0]*273
dfnight['it3'] = [0]*273+[1]*91+[0]*182
dfnight['it4'] = [0]*364+[1]*91+[0]*91
dfall = pd.concat([dfmorning, dfnight], axis=0)

In [50]:
dfall

,f0,f1,f2,f3,f4,f5,n0,n1,n2,n3,n4,n5,dn,it0,it1,it2,it3,it4
0,46.356000,48.588000,66.642500,64.681000,85.676000,58.968889,81.602857,80.207857,63.448462,78.051176,69.038333,69.657143,0,1,0,0,0,0
1,44.095000,59.874444,88.815000,77.847778,94.919167,73.677273,61.007000,332.162000,50.995000,44.935000,34.115000,23.295000,0,1,0,0,0,0
2,63.176667,87.030000,65.336667,78.012000,109.729333,114.720000,116.073636,125.105000,88.416842,71.472222,116.583333,88.655714,0,1,0,0,0,0
3,65.363333,57.038750,63.971429,132.218889,100.391765,65.348889,75.532500,77.430000,70.937500,69.421111,78.808889,61.732667,0,1,0,0,0,0
4,329.725833,80.130000,82.557333,87.626667,86.326667,96.311818,60.648750,89.380833,115.883636,109.430000,187.890556,151.746538,0,1,0,0,0,0
5,36.170000,63.176667,60.760000,106.390000,70.000000,63.798750,63.518571,62.114286,71.676000,60.730000,66.595000,83.738571,0,1,0,0,0,0
6,50.701250,40.385000,63.420000,85.789333,74.478462,141.304000,197.963125,199.165000,122.648824,81.544444,70.120000,72.356000,0,1,0,0,0,0
7,29.636667,83.015714,55.275556,64.600000,97.300000,98.635000,91.955333,79.574500,94.512941,120.258667,54.530909,76.308889,0,1,0,0,0,0
8,23.060000,77.396000,74.658000,91.734000,79.574167,93.998462,82.918333,123.381333,215.358824,86.847222,65.975333,52.546250,0,1,0,0,0,0
9,44.257500,55.885000,64.150000,56.910000,84.172222,88.321250,82.511765,78.567368,92.614000,78.451111,95.042646,111.634180,0,1,0,0,0,0


In [51]:
dfalltrain = dfall.iloc[:-84]
dfalltest = dfall.iloc[-84:]

In [52]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost

In [53]:
def set_pred_window(i):
    Y = dfalltrain.iloc[:,6+i]
    Yt = dfalltest.iloc[:,6+i]
    return Y, Yt

In [55]:
X = dfalltrain.iloc[:,[0,1,2,3,4,5,12,13,14,15,16,17]]
Xt = dfalltest.iloc[:,[0,1,2,3,4,5,12,13,14,15,16,17]]

In [81]:
Y, Yt = set_pred_window(1)

In [98]:
lm = LinearRegression(fit_intercept=False)
lm.fit(X, Y)
print pred_loss(lm.predict(X), Y)
print pred_loss(lm.predict(Xt), Yt)

0.251266122344
0.206930632392


In [113]:
lm = RandomForestRegressor(max_depth=7)
lm.fit(X, Y)
print pred_loss(lm.predict(X), Y)
print pred_loss(lm.predict(Xt), Yt)

0.190825250479
0.19121687925


In [115]:
lm = xgboost.XGBClassifier(max_depth=7)
lm.fit(X, Y)
print pred_loss(lm.predict(X), Y)
print pred_loss(lm.predict(Xt), Yt)

0.240201287085
0.331965929312


In [79]:
meandf = (dfalltest['f0']+dfalltest['f1']+dfalltest['f2']+
          dfalltest['f3']+dfalltest['f4']+dfalltest['f5'])/6

np.mean([pred_loss(meandf, dfalltest[idd]) 
         for idd in ['n1']])

0.16370041917290754

In [107]:
lm_list = []
for i in range(6):
    Y, Yt = set_pred_window(i)
    lm = RandomForestRegressor(max_depth=7)
    lm.fit(X, Y)
    #print pred_loss(lm.predict(X), Y)
    print pred_loss(lm.predict(Xt), Yt)
    lm_list.append(lm)

0.200233859041
0.150174223161
0.165474496801
0.185951746868
0.20708412761
0.196259154016


In [525]:
# lm = xgboost.XGBClassifier()
# lm.fit(X, Y)
# print pred_loss(lm.predict(X), Y)
# print pred_loss(lm.predict(Xt), Yt)

0.165805915328
0.262077128056


In [46]:
for i in lm_list:
    print i.coef_

[  5.04271725e-02   1.05088511e-04   8.62233252e-02   2.15531389e-01
   1.64171633e-01   2.89879520e-01  -6.00652007e+00]
[  0.08108744   0.03720535   0.05288547   0.10771723   0.18995688
   0.36242103 -10.13934917]
[  0.05288135   0.07684767   0.08300509   0.11372058   0.27232298
   0.17135667 -11.12290912]
[  1.19580810e-01   1.40978012e-02   9.96017860e-02   1.64229800e-01
   1.39465120e-01   2.43042507e-01  -1.45973813e+01]
[ 0.09874127  0.02823051  0.09975046  0.14426138  0.19544655  0.2089417
 -7.70160652]
[ 0.1547377  -0.02433399  0.11892897  0.19551693  0.16989831  0.10429231
 -1.2926539 ]


In [509]:
meandf = (dfall['f0']+dfall['f1']+dfall['f2']+
          dfall['f3']+dfall['f4']+dfall['f5'])/6

np.mean([pred_loss(meandf, dfall[idd]) 
         for idd in ['n0','n1','n2','n3','n4','n5']])

0.23057448974935238

In [510]:
meandf = (dfalltest['f0']+dfalltest['f1']+dfalltest['f2']+
          dfalltest['f3']+dfalltest['f4']+dfalltest['f5'])/6

np.mean([pred_loss(meandf, dfalltest[idd]) 
         for idd in ['n0','n1','n2','n3','n4','n5']])

0.1855935405015712

In [467]:
meandf = (dfmorning['f0']+dfmorning['f1']+dfmorning['f2']+dfmorning['f3']+dfmorning['f4']+dfmorning['f5'])/6

np.mean([pred_loss(meandf, dfmorning[idd]) for idd in ['n0','n1','n2','n3','n4','n5']])

0.24447079035997757

In [23]:
'a'.join(['b','c'])

'bac'

In [56]:
from scipy.stats import gmean

In [470]:
def sub1(x):
    return np.repeat([np.mean(i) for i in x], 6)

In [471]:
def sub2(x):
    return np.repeat([gmean(i) for i in x], 6)

In [24]:
def sos(x):
    return sum(i**2 for i in x)

In [25]:
def sl(x):
    return [i**2 for i in range(x)]

In [29]:
sos(sl(4))

98

In [25]:
def pred_loss(x_pred, x_true):
    x_pred = np.array(x_pred)
    x_true = np.array(x_true)
    return (abs(x_pred-x_true)/x_true).mean()

In [483]:
def true_value(date_list):
    morning_true = np.array([])
    for ii, ti in [('A',2),('A',3),('B',1),('B',3),('C',1),('C',3)]:
        mtmp = training_traj_group_mean.loc[ii].loc[ti]
        mtmp = interpolate_missing_window(mtmp)
        test6 = []
        for i in date_list:
            tmp = []
            for j in pred_time_list1:
                tmp.append(mtmp[datetime.combine(i, j)])
            test6.append(tmp)
        result6 = test6
        morning_true = np.append(morning_true, result6)
    return morning_true

In [484]:
def prediction(date_list, func):
    morning_pred = np.array([])
    for ii, ti in [('A',2),('A',3),('B',1),('B',3),('C',1),('C',3)]:
        mtmp = training_traj_group_mean.loc[ii].loc[ti]
        mtmp = interpolate_missing_window(mtmp)    
        test6 = []
        for i in date_list:
            tmp = []
            for j in give_time_list1:
                tmp.append(mtmp[datetime.combine(i, j)])
            test6.append(tmp)
        result6 = func(test6)
        morning_pred = np.append(morning_pred, result6) 
    return morning_pred

In [485]:
def sub3(x):
    return np.array([ma(i) for i in x]).flatten()

In [486]:
def ma(x):
    for i in range(6):
        x.append(np.mean(x[i:i+6]))
    return x[6:]

In [487]:
sub3([[1,2,3,4,5,6],[2,3,4,5,6,7]])

array([ 3.5       ,  3.91666667,  4.23611111,  4.44212963,  4.5158179 ,
        4.43512088,  4.5       ,  4.91666667,  5.23611111,  5.44212963,
        5.5158179 ,  5.43512088])

In [488]:
morning_true = true_value(date_list)
morning_pred = prediction(date_list, sub1)


In [489]:
pred_loss(morning_pred, morning_true)

0.24447079035997757

In [31]:
morning_train66 = []
for i in date_list:
    tmp = []
    for j in give_time_list1:
        tmp.append(A2m[datetime.combine(i, j)])
    for j in pred_time_list1:
        tmp.append(A2m[datetime.combine(i, j)]) 
    morning_train66.append(tmp)
    

In [49]:
night_train66 = []
for i in date_list:
    tmp = []
    for j in give_time_list2:
        tmp.append(A2m[datetime.combine(i, j)])
    for j in pred_time_list2:
        tmp.append(A2m[datetime.combine(i, j)]) 
    night_train66.append(tmp)
    

In [34]:
def naive_pred_loss(x):
    first6 = x[:6]
    pred = [np.mean(first6)]*6
    return pred_loss(pred, x[6:])

In [35]:
naive_loss_list = [naive_pred_loss(i) for i in morning_train66]


In [50]:
naive_loss_list = [naive_pred_loss(i) for i in night_train66]

In [52]:
np.mean(naive_loss_list)


0.22280558325098837

# Make first submission 

Most naive approach, use average of the former 2 hours for prediction.

Change algebraic average to geometric average for second submission.

In [680]:
#training_traj['starting_datetime'] = pd.to_datetime(training_path['starting_time'])
training_traj['window'] = training_traj['starting_time'].apply(time_to_window)

training_traj_group_mean = \
training_traj.groupby(['intersection_id','tollgate_id','window']).mean()['travel_time']

training_traj_group_count = \
training_traj.groupby(['intersection_id','tollgate_id','window']).count()['travel_time']

In [681]:
test_traj1['window'] = test_traj1['starting_time'].apply(time_to_window)
test_traj1_group_mean = \
test_traj1.groupby(['intersection_id','tollgate_id','window']).mean()['travel_time']



In [682]:
date_list = []

d1 = date(2016, 10, 18)  # start date
d2 = date(2016, 10, 24)  # end date

delta = d2 - d1         # timedelta

for i in range(delta.days + 1):
    date_list.append(d1 + timedelta(days=i))

give_time_list1 = [time(6,0), time(6,20), time(6,40), 
                   time(7,0), time(7,20), time(7,40)]
# pred_time_list1 = [time(8,0), time(8,20), time(8,40), 
#                    time(9,0), time(9,20), time(9,40)]
give_time_list2 = [time(15,0), time(15,20), time(15,40), 
                   time(16,0), time(16,20), time(16,40)]

In [683]:
A2mt = test_traj1_group_mean.loc['A'].loc[2]
A2mt = interpolate_missing_window(A2mt)

In [684]:
morning_test6 = []
for i in date_list:
    tmp = []
    for j in give_time_list1:
        tmp.append(A2mt[datetime.combine(i, j)])
    morning_test6.append(tmp)

In [685]:
tmp

[41.18833333333334,
 50.115714285714283,
 37.168000000000006,
 75.683750000000003,
 54.032857142857146,
 78.277999999999992]

In [694]:
test_traj1_group_mean.loc['A'].loc[2].iloc[:6]

window
2016-10-18 06:00:00    41.097143
2016-10-18 06:20:00    43.681667
2016-10-18 06:40:00    68.024286
2016-10-18 07:00:00    52.608333
2016-10-18 07:20:00    56.165556
2016-10-18 07:40:00    63.598571
Name: travel_time, dtype: float64

In [691]:
lm_list[0].coef_

array([  5.04271725e-02,   1.05088511e-04,   8.62233252e-02,
         2.15531389e-01,   1.64171633e-01,   2.89879520e-01,
        -6.00652007e+00])

In [692]:
lm_list[0].intercept_

30.893025008264075

In [686]:
def predict_from_lm(t6, dn):
    return [lm_list[i].predict(np.array(t6+[dn]).reshape(1,-1))[0] \
            for i in range(6)]

In [687]:
morning = np.array([])
for ii, ti in [('A',2),('A',3),('B',1),('B',3),('C',1),('C',3)]:
    mtmp = test_traj1_group_mean.loc[ii].loc[ti]
    mtmp = interpolate_missing_window(mtmp)
    test6 = []
    for i in date_list:
        tmp = []
        for j in give_time_list1:
            tmp.append(mtmp[datetime.combine(i, j)])
        test6.append(tmp)
    result6 = [predict_from_lm(i, 0) for i in test6]
    result6 = [item for sublist in result6 for item in sublist]
    #result6 = np.repeat([gmean(i) for i in test6], 6)
    morning = np.append(morning, result6)
    

In [688]:
night = np.array([])
for ii, ti in [('A',2),('A',3),('B',1),('B',3),('C',1),('C',3)]:
    mtmp = test_traj1_group_mean.loc[ii].loc[ti]
    mtmp = interpolate_missing_window(mtmp)
    test6 = []
    for i in date_list:
        tmp = []
        for j in give_time_list2:
            tmp.append(mtmp[datetime.combine(i, j)])
        test6.append(tmp)
    result6 = [predict_from_lm(i, 1) for i in test6]
    result6 = [item for sublist in result6 for item in sublist]
    #result6 = np.repeat([gmean(i) for i in test6], 6)
    night = np.append(night, result6)

In [689]:
ss['avg_travel_time'] = np.append(morning, night)

In [690]:
ss.head(20)

,intersection_id,tollgate_id,time_window,avg_travel_time
0,A,2,"[2016-10-18 08:00:00,2016-10-18 08:20:00)",77.830770
1,A,2,"[2016-10-18 08:20:00,2016-10-18 08:40:00)",80.071507
2,A,2,"[2016-10-18 08:40:00,2016-10-18 09:00:00)",86.402262
3,A,2,"[2016-10-18 09:00:00,2016-10-18 09:20:00)",89.841479
4,A,2,"[2016-10-18 09:20:00,2016-10-18 09:40:00)",88.785946
5,A,2,"[2016-10-18 09:40:00,2016-10-18 10:00:00)",86.286607
6,A,2,"[2016-10-19 08:00:00,2016-10-19 08:20:00)",84.473507
7,A,2,"[2016-10-19 08:20:00,2016-10-19 08:40:00)",87.113912
8,A,2,"[2016-10-19 08:40:00,2016-10-19 09:00:00)",97.817200
9,A,2,"[2016-10-19 09:00:00,2016-10-19 09:20:00)",94.477799


In [696]:
ss.to_csv('submission/submission_0403.csv', index=False)

# Following is the script given

In [18]:
# -*- coding: utf-8 -*-
#!/usr/bin/env python

"""
Objective:
Calculate the average travel time for each 20-minute time window.

"""

# import necessary modules
import math
from datetime import datetime,timedelta

file_suffix = '.csv'
path = 'dataSets/training/'  # set the data directory

def avgTravelTime(in_file):

    out_suffix = '_20min_avg_travel_time'
    in_file_name = in_file + file_suffix
    out_file_name = in_file.split('_')[1] + out_suffix + file_suffix

    # Step 1: Load trajectories
    fr = open(path + in_file_name, 'r')
    fr.readline()  # skip the header
    traj_data = fr.readlines()
    fr.close()
    print(traj_data[0])

    # Step 2: Create a dictionary to store travel time for each route per time window
    travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
    for i in range(len(traj_data)):
        each_traj = traj_data[i].replace('"', '').split(',')
        intersection_id = each_traj[0]
        tollgate_id = each_traj[1]

        route_id = intersection_id + '-' + tollgate_id
        if route_id not in travel_times.keys():
            travel_times[route_id] = {}

        trace_start_time = each_traj[3]
        trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
        time_window_minute = int(trace_start_time.minute / 20) * 20
        start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     trace_start_time.hour, time_window_minute, 0)
        tt = float(each_traj[-1]) # travel time

        if start_time_window not in travel_times[route_id].keys():
            travel_times[route_id][start_time_window] = [tt]
        else:
            travel_times[route_id][start_time_window].append(tt)

    # Step 3: Calculate average travel time for each route per time window
    fw = open(out_file_name, 'w')
    fw.writelines(','.join(['"intersection_id"', '"tollgate_id"', '"time_window"', '"avg_travel_time"']) + '\n')
    for route in travel_times.keys():
        route_time_windows = list(travel_times[route].keys())
        route_time_windows.sort()
        for time_window_start in route_time_windows:
            time_window_end = time_window_start + timedelta(minutes=20)
            tt_set = travel_times[route][time_window_start]
            avg_tt = round(sum(tt_set) / float(len(tt_set)), 2)
            out_line = ','.join(['"' + route.split('-')[0] + '"', '"' + route.split('-')[1] + '"',
                                 '"[' + str(time_window_start) + ',' + str(time_window_end) + ')"',
                                 '"' + str(avg_tt) + '"']) + '\n'
            fw.writelines(out_line)
    fw.close()

def main():

    in_file = 'trajectories(table 5)_training'
    avgTravelTime(in_file)

if __name__ == '__main__':
    main()





"B","3","1065642","2016-07-19 00:14:24","105#2016-07-19 00:14:24#9.56;100#2016-07-19 00:14:34#6.75;111#2016-07-19 00:14:41#13.00;103#2016-07-19 00:14:54#7.47;122#2016-07-19 00:15:02#32.85","70.85"

